## Bergfex Webscraping
<b> Data Preparation</b> 

In previous notebooks (part I notebooks 1-3), we scraped the activity data and the weather data (i.e. snow levels). 

Most visualizations we can directly build in Tableau without further data manipulation.
For one plot, we like to take advantage of the true meaning of ascent, i.e. the gain in height meters during most mountain activities. To compare the relative steepness of the different activities we will build a line chart where the slope will represent the steepness. Therefore, we need to create an additional data frame.

### Imports

In [1]:
import pandas as pd

# Import prepared files

In [2]:
# change the file_path to your path if necessary
filepath = '../data/'

# importing activity data
activities = pd.read_csv(filepath + 'activities_all_gpx.csv')

# Data manipulation
This additional DF should have the starting altitude and the maximum altitude in the same column (previously in separate columns 'min' and 'max'). 
Another trick to tackle the handling of hours/min in Tableau is to convert the duration to a decimal number, which is still easily understood (i.e. from '01:30' to '1.5').

## Create new DF

In [3]:
min_gpx = activities.loc[:,['ID','min']]                  # create DF for starting altitude
min_gpx.columns = ['ID','altitude_in_m']                  # rename columns 
min_gpx['min_max'] = 'min'                                # create new column with "min" as value
min_gpx['time'] = '00:01'                                 # create new column with "00:01" as start time value
max_gpx = activities.loc[:,['ID','max','time']]           # create the same for the max time
max_gpx.columns = ['ID','altitude_in_m','time']
max_gpx['min_max'] = 'max'
new_gpx = pd.concat([min_gpx, max_gpx], axis = 0)         # append DF under each other
new_gpx.shape, new_gpx.head()

((7592, 4),
        ID  altitude_in_m min_max   time
 0  129549          392.0     min  00:01
 1  129607          391.0     min  00:01
 2  961881          670.0     min  00:01
 3  129624          395.0     min  00:01
 4  897134          438.0     min  00:01)

## Adjust the duration to a decimal time

First, we will adjust the duration to a decimal number.

Second, we need to adjust the duration it takes to reach the peak. We will use an assumption: Normally the descent of an activity takes less time than the ascent, let's say 2/3 of the total duration we need for the ascent. Of course, this is subjective and it also depends on the specific activity. The descent on a mountain bike or on skis could easily be much quicker than 1/3 of the duration. For simplicity, however, we keep the same assumption across all activity types.

In [4]:
new_gpx['time'] = pd.Series(new_gpx['time']).str.replace("-", '00:01') # harmonize time formatting
new_gpx1 = new_gpx['time'].str.split(':',expand = True)   # split time into 2 columns
new_gpx1[0] = pd.to_numeric(new_gpx1[0], errors='coerce') # convert to int
new_gpx1[1] = pd.to_numeric(new_gpx1[1], errors='coerce') # convert to int
new_gpx1['sec'] = round(new_gpx1[1]/60,1)                 # convert seconds into decimals
new_gpx1['time'] = new_gpx1[0] + new_gpx1['sec']          # add minutes and seconds again
new_gpx['time_decimals'] = new_gpx1['time']               # add as new column
new_gpx['time_peak'] = new_gpx['time_decimals'] * 2/3     # Assumption: ascent takes 2/3 of the total time
new_gpx['time_peak'] = round(new_gpx['time_peak'],1)
#new_gpx = new_gpx.drop(columns = ['time'])                # drop old column
new_gpx.tail()

,ID,altitude_in_m,min_max,time,time_decimals,time_peak
3791,130890,3030.0,max,01:15,1.2,0.8
3792,513541,2125.0,max,01:30,1.5,1.0
3793,130940,2253.0,max,02:30,2.5,1.7
3794,130882,3196.0,max,01:30,1.5,1.0
3795,183267,1605.0,max,03:00,3.0,2.0


# Write CSV file

In [5]:
new_gpx.to_csv(filepath + 'rearranged_all.csv', index = False)